In [1]:
from eval_cuhk03_set import *

In [3]:
test_list=range(1,2) #use set 1-10 for test (total 20)
cmc_list=[]
for set_no in test_list:
    #init net
    for i in range(0,1):
        MODEL_FILE = '../../experiments/reid_earlyfusion_imgset/set%02d/'%(set_no)+'deploy.prototxt'
        PRETRAINED = '../../experiments/reid_earlyfusion_imgset/set%02d/'%(set_no)+'Snapshots/set%02d_iter_%d.caffemodel'%(set_no,(i+1)*5000)
        caffe.set_device(0)
        caffe.set_mode_gpu()
        net = caffe.Classifier(MODEL_FILE, PRETRAINED,caffe.TEST)
        #caculate CMC
        cmc=calCMC(net,set_no,rand_times=1)
        print cmc
        cmc_list.append(cmc)

../../dataset/cuhk03/cuhk03_release/exp_set/set01_test_noval.txt

Round 0 with rand list:
  Processing 100x100 pairs cost 127.214580 second time
[ 0.24  0.39  0.54  0.59  0.64  0.68  0.71  0.73  0.74  0.75  0.75  0.79
  0.79  0.79  0.79  0.81  0.82  0.82  0.83  0.83  0.85  0.86  0.87  0.87
  0.87  0.87  0.87  0.87  0.87  0.87  0.87  0.87  0.87  0.89  0.9   0.9
  0.9   0.9   0.9   0.9   0.9   0.92  0.93  0.93  0.93  0.93  0.95  0.96
  0.96  0.96  0.96  0.96  0.96  0.97  0.97  0.97  0.97  0.97  0.97  0.98
  0.98  0.98  0.98  0.98  0.98  0.98  0.98  0.99  0.99  0.99  0.99  0.99
  0.99  0.99  0.99  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
  1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
  1.    1.  ]


In [ ]:
test_list=range(1,2) #use set 1-10 for test (total 20)
cmc_list=[]
for set_no in test_list:
    #init net
    for i in range(16):
        MODEL_FILE = '../../experiments/reid_earlyfusion_drop10//set%02d/'%(set_no)+'deploy.prototxt'
        PRETRAINED = '../../experiments/reid_earlyfusion_drop10/set%02d/'%(set_no)+'Snapshots/set%02d_iter_%d.caffemodel'%(set_no,(i+1)*5000)
        caffe.set_device(0)
        caffe.set_mode_gpu()
        net = caffe.Classifier(MODEL_FILE, PRETRAINED,caffe.TEST)
        #caculate CMC
        cmc=calCMC(net,set_no,rand_times=10)
        print cmc
        cmc_list.append(cmc)

In [ ]:
#dict of cmc, each value is a cmc list (at least 50 dim vetor, rank 1 to rank 50)
cmcDict={}
cmcDict['CVPR15Imp']=getCVPRcmc()
cmcDict['OurMethod']=np.average(cmc_list[:],axis=0)

#plot the dictionary, sorted by rank1 rate
plotCMC(cmcDict)

In [ ]:
print np.average(cmc_list[8:],axis=0)

In [ ]:
dict1={}
dict1['1']=['hello']
dict1['1'].append([1,2,3,4])
dict1['2']=['dsvhello']
dict1['2'].append([1,2,3,4,5])

In [ ]:
print dict1.items()

In [ ]:
def plotCMC(cmcDict):
    import matplotlib.pyplot as plt
    get_ipython().magic(u'matplotlib inline')
    from matplotlib.legend_handler import HandlerLine2D
    import numpy as np

    #plot the cmc curve, record CVPR from the pdf paper.cmc[0,4,8,12,16,21,25,29,33,37,41,45,49]
    cmcIndex=[0,4,8,12,16,21,25]
    colorList=['g','b','r','y','c','m','k']
    #start to plot
    plt.figure(figsize=(12,8),dpi=180)
    sortedCmcDict = sorted(cmcDict.items(), key=lambda (k, v): v[1])[::-1]
    for idx in range(len(sortedCmcDict)):
        cmc_dictList=sortedCmcDict[idx]
        cmc_name=cmc_dictList[0]
        cmc_list=cmc_dictList[1]
        #print cmc_name,": ",cmc_list
        #x for plot
        x_point=[item+1 for item in cmcIndex[0:-1]]
        x_line=range(25)
        x_plot=[temp+1 for temp in x_line]
        #start plot
        plt.plot(x_plot, cmc_list[x_line],colorList[idx],label="%02.02f%% %s"%(100*cmc_list[0],cmc_name))
        plt.plot(x_point,cmc_list[cmcIndex[0:-1]],colorList[idx]+'*')
        #plt.legend(loc=4,handler_map={line: HandlerLine2D(numpoints=1)})
        #idx of color +1
        idx+=1
    plt.xlabel('Rank')
    plt.ylabel('Identification Rate')
    plt.xticks(np.arange(0,26,5))
    plt.yticks(np.arange(0.5,1.01,0.1))
    plt.grid()
    plt.legend(loc=4)
    plt.show()

# Just for Debug

In [ ]:
set_no=3
#init net
MODEL_FILE = '../../experiments/reid_simplemodel/set%02d/'%(set_no)+'deploy.prototxt'
PRETRAINED = '../../experiments/reid_simplemodel/set%02d/'%(set_no)+'Snapshots/set%02d_iter_120000.caffemodel'%(set_no)
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Classifier(MODEL_FILE, PRETRAINED,caffe.TEST)
list_name=DATA_DIR+'exp_set/set%02d_test_noval.txt'%(set_no)
print list_name
probes,gallerys=readList(list_name)

#for only one probe test
transformer = caffe.io.Transformer({'data': (net.blobs['data'].data.shape)})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.array([ 104.00698793,  116.66876762,  122.67891434])) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

scoreList=[]
for probeName in probes[0:1]:
    probeImage=transformer.preprocess('data', caffe.io.load_image(probeName))
    batchSize= 100
    C,H,W=probeImage.shape
    probeData=np.zeros((batchSize,C,H,W))
    probeData[:,:,:,:]=probeImage
    galleryIdx=0
    probeScoreLists=[]
    while galleryIdx<len(gallerys):
        galleryDataList=[]
        for batchIdx in range(batchSize):
            if galleryIdx>=len(gallerys):
                break
            else:
                galleryName=gallerys[galleryIdx]
                galleryImage=transformer.preprocess('data', caffe.io.load_image(galleryName))
                galleryDataList.append(galleryImage)
                galleryIdx+=1
        galleryData=np.asarray(galleryDataList)
        #init net data with images
        N,C,H,W=galleryData.shape
        net.blobs['data'].reshape(N,C,H,W)
        net.blobs['data_p'].reshape(N,C,H,W)
        net.blobs['data'].data[:] = probeData[0:N,:]
        net.blobs['data_p'].data[:] = galleryData
        #net forwad
        net.forward()
        #get output score
        outScore=net.blobs['softmax_score'].data[:,(0,1)]    #softmax_score[0] and softmax_score[1]
        #score_sum=np.exp(outScore[:,0]*1.0)+np.exp(outScore[:,1]*1.0)
        similarScore=outScore[:,1]    #np.exp(outScore[:,1]*1.0)/score_sum
        probeScoreLists.extend(similarScore.tolist())
    scoreList.append(probeScoreLists)
    print probeName
predictLists=[]